<a href="https://colab.research.google.com/github/Lornng/FineTune/blob/main/Merge_model_200_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31 #temporary fix required owing to breaking changes on Aug 9th 2023
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load and quantize base model

In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

#Load and quantize the model
base_model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
base_model.config.use_cache = False


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load the adapter model and save the model
Hugging face doesnt support 4-bit model

In [4]:
from peft import PeftModel

# Load the adapter model
adapter_model = PeftModel.from_pretrained( model=base_model , model_id="Lornng/Llama2-7B-QLoRA-cpgQA", adapter_name="Lornng/Llama2-7B-QLoRA-cpgQA")

In [5]:
# Merge the quantized base model and the adapter model
merged_model = adapter_model.merge_and_unload(base_model)

Unloading and merging model:   0%|          | 0/518 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:213: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Unloading and merging model: 100%|██████████| 518/518 [00:09<00:00, 56.53it/s]


In [13]:
# Push the merged model to the Hugging Face Hub

# merged_model.push_to_hub("Llama-2-7b-merged-QLoRA-cpgQA")

# hugging face does not support 4-bit model to be uplaoded

In [6]:
# Save the merged model as a TorchScript module
script_model = torch.jit.script(merged_model)
torch.jit.save(script_model, "Llama-2-7b-merged-QLoRA-cpgQA.pt")

AttributeError: ignored